# Importing and Utils

In [1]:
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

from torch.optim import Adam


random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
environment = "kaggle"

if environment == "local":
  data_root = "data"
  BATCH_SIZE = 4
elif environment == "kaggle":
  data_root = "/kaggle/input/marquis-viton-hd"
  BATCH_SIZE = 16

train_folder = os.path.join(data_root, "train")
valid_folder = os.path.join(data_root, "test")

train_file_names = os.listdir(os.path.join(train_folder, "image"))
random.shuffle(train_file_names)

valid_file_names = os.listdir(os.path.join(valid_folder, "image"))
random.shuffle(valid_file_names)

In [3]:
import wandb

LR = 1e-4
EPOCHS = 20
experiment_name = "v0-baseline-0.3"

if environment == "kaggle":
    from kaggle_secrets import UserSecretsClient
    
    user_secrets = UserSecretsClient()
    wandb_api = user_secrets.get_secret("WANDB_API_KEY")
    
    wandb.login(key=wandb_api)

wandb.init(
    project="viton",
    name=experiment_name,
    tags=["torch", environment, "P100", "1GPU"],
    notes="Large Model (1024 Bottleneck + Batch Norm), Smaller Image (288, 384)",
  
    config={
    "learning_rate": LR,
    "architecture": "UNet",
    "epochs": EPOCHS,
    }
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sashanktalakola2 (sashanktalakola). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240607_035513-pd0tzdee
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run v0-baseline-0.3
wandb: ⭐️ View project at https://wandb.ai/sashanktalakola/viton
wandb: 🚀 View run at https://wandb.ai/sashanktalakola/viton/runs/pd0tzdee


# Dataset

In [4]:
IMG_SIZE = (288, 384)

train_transforms = A.Compose([
    A.Resize(height=IMG_SIZE[0], width=IMG_SIZE[1]),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

label_transforms = A.Compose([
  A.Resize(height=IMG_SIZE[0], width=IMG_SIZE[1]),
  ToTensorV2()
])

valid_transforms = A.Compose([
    A.Resize(height=IMG_SIZE[0], width=IMG_SIZE[1]),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [5]:
class TrainDataset(Dataset):
  def __init__(self, image_list, image_folder, transforms, label_transforms):
    self.image_list = image_list
    self.image_folder = image_folder
    self.transforms = transforms
    self.label_transforms = label_transforms

  def __len__(self): return len(self.image_list)

  def __getitem__(self, i):
    image_name = self.image_list[i]

    agnostic_path = os.path.join(self.image_folder, "agnostic", image_name)
    cloth_path = os.path.join(self.image_folder, "cloth", image_name)
    output_img_path = os.path.join(self.image_folder, "image", image_name)

    agnostic_image = cv2.cvtColor(cv2.imread(agnostic_path), cv2.COLOR_BGR2RGB)
    cloth_image = cv2.cvtColor(cv2.imread(cloth_path), cv2.COLOR_BGR2RGB)
    output_img_image = cv2.cvtColor(cv2.imread(output_img_path), cv2.COLOR_BGR2RGB)

    agnostic_image = self.transforms(image=agnostic_image)["image"]
    cloth_image = self.transforms(image=cloth_image)["image"]
    output_img_image = self.label_transforms(image=output_img_image)["image"]

    return agnostic_image, cloth_image, output_img_image.float()

train_dataset = TrainDataset(train_file_names, train_folder, train_transforms, label_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class ValidDataset(Dataset):
  def __init__(self, image_list, image_folder, transforms, label_transforms):
    self.image_list = image_list
    self.image_folder = image_folder
    self.transforms = transforms
    self.label_transforms = label_transforms

  def __len__(self): return len(self.image_list)

  def __getitem__(self, i):
    image_name = self.image_list[i]

    agnostic_path = os.path.join(self.image_folder, "agnostic", image_name)
    cloth_path = os.path.join(self.image_folder, "cloth", image_name)
    output_img_path = os.path.join(self.image_folder, "image", image_name)

    agnostic_image = cv2.cvtColor(cv2.imread(agnostic_path), cv2.COLOR_BGR2RGB)
    cloth_image = cv2.cvtColor(cv2.imread(cloth_path), cv2.COLOR_BGR2RGB)
    output_img_image = cv2.cvtColor(cv2.imread(output_img_path), cv2.COLOR_BGR2RGB)

    agnostic_image = self.transforms(image=agnostic_image)["image"]
    cloth_image = self.transforms(image=cloth_image)["image"]
    output_img_image = self.label_transforms(image=output_img_image)["image"]

    return agnostic_image, cloth_image, output_img_image.float()

valid_dataset = TrainDataset(valid_file_names, valid_folder, train_transforms, label_transforms)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model

In [7]:
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
    super(DoubleConv, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.padding = padding

    self.double_conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

  def forward(self, X):
    X = self.double_conv(X)
    
    return X

In [8]:
class DownSample(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
    super(DownSample, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.padding = padding

    self.double_conv = DoubleConv(in_channels, out_channels, kernel_size=kernel_size, padding=padding)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

  def forward(self, X):
    X = self.double_conv(X)
    X_pooled = self.pool(X)

    return X, X_pooled

In [9]:
class UpSample(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
    super(UpSample, self).__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.padding = padding

    self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
    self.double_conv = DoubleConv(in_channels, out_channels, kernel_size=kernel_size, padding=padding)

  def forward(self, X, X_skip):
    X = self.up(X)

    X_cat = torch.cat((X, X_skip), dim=1)
    X_cat = self.double_conv(X_cat)

    return X_cat

In [10]:
class UNet(nn.Module):
  def __init__(self, filter_sizes):
    super(UNet, self).__init__()

    self.filter_sizes = filter_sizes
    self.down_sample_blocks = []

    current_in_channels = 6
    for filter_size in filter_sizes:
      self.down_sample_blocks.append(DownSample(current_in_channels, filter_size, kernel_size=3, padding=1).to(device))
      current_in_channels = filter_size

    self.bottleneck = DoubleConv(filter_sizes[-1], filter_sizes[-1]*2, kernel_size=3, padding=1)

    self.up_sample_blocks = []
    for filter_size in filter_sizes[::-1]:
      self.up_sample_blocks.append(UpSample(filter_size*2, filter_size, kernel_size=3, padding=1).to(device))

    self.out_conv = nn.Conv2d(filter_sizes[0], 3, kernel_size=1)


  def forward(self, X_agnostic, X_cloth):
    X = torch.cat((X_agnostic, X_cloth), dim=1)

    X1_skip, X = self.down_sample_blocks[0](X)
    X2_skip, X = self.down_sample_blocks[1](X)
    X3_skip, X = self.down_sample_blocks[2](X)
    X4_skip, X = self.down_sample_blocks[3](X)

    X = self.bottleneck(X)

    X = self.up_sample_blocks[0](X, X4_skip)
    X = self.up_sample_blocks[1](X, X3_skip)
    X = self.up_sample_blocks[2](X, X2_skip)
    X = self.up_sample_blocks[3](X, X1_skip)

    X = self.out_conv(X)

    return X

In [11]:
model = UNet([64, 128, 256, 512]).to(device)

# Training

## Train Loop

In [12]:
loss_fn = torch.nn.MSELoss()
optimizer = Adam(model.parameters(), lr=LR)

In [13]:
from tqdm import tqdm

def train(model, dataloader, loss_fn, optimizer, epoch):
  model.train()

  batch_losses = []

  pbar = tqdm(dataloader, unit="batch", leave=True, desc=f"Training : Epoch [{epoch+1}/{EPOCHS}]")
  for batch_idx, (X1, X2, target) in enumerate(pbar):
    X1 = X1.to(device)
    X2 = X2.to(device)
    target = target.to(device)

    prediction = model(X1, X2)
    loss = loss_fn(prediction, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    batch_losses.append(loss.item())
    pbar.set_postfix({"Batch Loss": loss.item()})

  return batch_losses

## Validation Loop

In [14]:
def valid(model, dataloader, loss_fn, epoch):
  model.eval()

  batch_losses = []

  pbar = tqdm(dataloader, unit="batch", leave=True, desc=f"Validation : Epoch [{epoch+1}/{EPOCHS}]")
  for batch_idx, (X1, X2, target) in enumerate(pbar):
    X1 = X1.to(device)
    X2 = X2.to(device)
    target = target.to(device)

    with torch.inference_mode():
      prediction = model(X1, X2)
      loss = loss_fn(prediction, target)

    batch_losses.append(loss.item())
    pbar.set_postfix({"Batch Loss": loss.item()})

  return batch_losses

In [15]:
os.makedirs(f"saved-models/{experiment_name}", exist_ok=True)

for epoch in range(EPOCHS):
  train_losses = train(model, train_dataloader, loss_fn, optimizer, epoch)
  train_epoch_loss = sum(train_losses) / len(train_losses)
  print(f"Epoch [{epoch+1}/{EPOCHS}]\tLoss: {train_epoch_loss}")

  valid_losses = valid(model, valid_dataloader, loss_fn, epoch)
  valid_epoch_loss = sum(valid_losses) / len(valid_losses)
  print(f"Epoch [{epoch+1}/{EPOCHS}]\tLoss: {valid_epoch_loss}\n")

  wandb.log({"train_epoch_loss": train_epoch_loss, "valid_epoch_loss": valid_epoch_loss})
  torch.save(model.state_dict(), f"saved-models/epoch - {epoch}")

wandb.finish()

Training : Epoch [1/20]: 100%|██████████| 728/728 [20:01<00:00,  1.65s/batch, Batch Loss=3.75e+4]


Epoch [1/20]	Loss: 38216.4814453125


Validation : Epoch [1/20]: 100%|██████████| 127/127 [02:31<00:00,  1.19s/batch, Batch Loss=4e+4]


Epoch [1/20]	Loss: 37888.46465920276



Training : Epoch [2/20]: 100%|██████████| 728/728 [14:46<00:00,  1.22s/batch, Batch Loss=3.68e+4]


...

Training : Epoch [20/20]: 100%|██████████| 728/728 [14:56<00:00,  1.23s/batch, Batch Loss=2.75e+4]


Epoch [20/20]	Loss: 28472.257689088256


Validation : Epoch [20/20]: 100%|██████████| 127/127 [01:40<00:00,  1.26batch/s, Batch Loss=2.99e+4]


Epoch [20/20]	Loss: 28214.060254675198



wandb:                                                                                
wandb: 
wandb: Run history:
wandb: train_epoch_loss ██▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁
wandb: valid_epoch_loss ██▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁
wandb: 
wandb: Run summary:
wandb: train_epoch_loss 28472.25769
wandb: valid_epoch_loss 28214.06025
wandb: 
wandb: 🚀 View run v0-baseline-0.3 at: https://wandb.ai/sashanktalakola/viton/runs/pd0tzdee
wandb: ⭐️ View project at: https://wandb.ai/sashanktalakola/viton
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240607_035513-pd0tzdee/logs
